# Artificial Intelligence - Project 1

#### importing

In [104]:
import pandas as pd
import os
import csv
import numpy as np
import time
from collections import deque
import bisect
import math
from statistics import mean

#### reading the csv file

In [303]:
file_path = r'C:\Users\pr\test_03.txt'
time0 = time.time()

with open(file_path, newline='') as f:
    reader = csv.reader(f, delimiter=" ")
    row, col = list(map(int, next(reader)))
    strt = list(map(int, next(reader)))
    dest = list(map(int, next(reader)))
    orc_no, fellow_no = list(map(int, next(reader)))
    
    orc_data = np.zeros((int(orc_no), 3)).tolist()
    for i in range(orc_no): 
        orc_data[i] = list(map(int, next(reader)))
    
    fellow_strt = np.zeros((fellow_no, 2)).tolist()
    for i in range(fellow_no): 
        fellow_strt[i] = list(map(int, next(reader)))
        
    fellow_dest = np.zeros((fellow_no, 2)).tolist()
    for i in range(fellow_no): 
        fellow_dest[i] = list(map(int, next(reader)))
        
        
time1 = time.time()
print('time consumed to perform this task is {}'.format(time1-time0))

time consumed to perform this task is 0.0


note: the heuristic function is declared here to be usable inside the __lt__ function and is later explained

In [318]:
def heuristic(current_state):
    
    h = 0 
    
    current_to_member_strt_min = row + col
    member_dest_to_Gondor_min = row + col
    member_state_2 = []
    
    origin_of_calc = current_state[0:2]
    
    for i in range(fellow_no):
        
        if current_state[2+i] == 2:
            member_state_2.append(i)
            h += manhattan(fellow_dest[i], fellow_strt[i])
            if manhattan(fellow_dest[i], goal_state[0:2]) <= member_dest_to_Gondor_min:
                member_dest_to_Gondor_min = manhattan(fellow_dest[i], goal_state[0:2])
            
        if current_state[2+i] == 1:
            member_state_1 = i
            origin_of_calc = fellow_dest[i]
            h += manhattan(current_state[0:2], fellow_dest[i])
    
    for m in member_state_2:
        if manhattan(fellow_strt[i], origin_of_calc) <= current_to_member_strt_min:
            current_to_member_strt_min = manhattan(fellow_strt[i], origin_of_calc) 
        
    
    if current_to_member_strt_min != row + col:
        h += current_to_member_strt_min
    if member_dest_to_Gondor_min != row + col:
        h += member_dest_to_Gondor_min
        
    return h*1.5

### How to tackle the problem

First we need to create a model of our game.

##### States (How to describe the game status)

In order to have a completely descriptive of our state, we should know to following:
    - the coordinates of Gandalf
    - the state of each member of fellowship (starting point/with Gandalf/destination)
    - the steps taken in the space guarded by each orc
    
For instance, the initial state of the example in the pdf file will be as shown bellow:
    - state = Gandalf_Coordinates + Conditions_of_Fellow_Members + Steps_in_Orc_Zones = [0, 0] + [2, 2] + [0, 0, 0]

##### Actions

The possible actions are up, down, right and left, unless:
    - the action moves Gandalf out of the grid
    - the action is the (n+1)th step inside an orc's guarded area
    
    
##### Goal

Gandalf and each member of the fellowship should have the same coordinate as their destination.In other words, all fellowship members should be in state 0 (Similar to steps taken in orc zones.) For example, the goal state for our pdf example will be as follows:
    - goal_state = [7, 7, 0, 0, 0, 0, 0]


##### Cost

The cost of each move should be equal to one as there are no priorities mentioned in problem.

________________________

#### Creating the Node Class

    - state, parent, path, path_cost, depth, and map are class variables.
    - the equality of Nodes are examined by comparing their map which is created from converting Node.state to string
    - hash function is used for keeping the Nodes in a set when listing them makes the program slow
    - __lt__ function is used for A* search algorithm. It is used in inserting a new Node in a sorted list of Nodes
    - path_str() function is used to print the path taken to the goal_state


In [306]:
class Node:
    def __init__(self, state, parent, path_cost, path):
        self.state = state
        self.parent = parent
        self.path_cost = path_cost
        self.path = path
        self.depth = len(self.path)
        
        if self.state:
            self.map = ''.join(str(e) for e in self.state)
            
    def __eq__(self, other):
        return self.map == other.map
    
    def __hash__(self):
        return hash(self.map)
    
    def __len__(self):
        return self.depth
    
    def __lt__(self, other):
        if self.path_cost + heuristic(self.state) < other.path_cost + heuristic(other.state):
            return True
        return False
    
    def path_str(self):
        string_path = ''
        for i in self.path:
            if i==0: string_path += 'U'
            elif i==1: string_path += 'D'
            elif i==2: string_path += 'R'
            elif i==3: string_path += 'L'
                
        return string_path

#### initial and final states

the conditions of members of the fellowship are coded as follows:
    - at the starting point = 2
    - Acompanied by Gandalf = 1
    - at the destination = 0
    
The initial state is (strt + [2] * fellow_no + [0] * orc_no) because at the begining all the members of fellowship are at the starting point (which, as told, is not guarded by any orc)

The final state is (dest + [0] * fellow_no + [0] * orc_no) because at the end all the members of fellowship are at the destination (which, as told, is not guarded by any orc)

________________________

##### Defining the initial and final states and useful functions

In [307]:
initial_state = strt + [2] * fellow_no + [0] * orc_no
goal_state = dest + [0] * fellow_no + [0] * orc_no

initial_node = Node(initial_state, None, 0, [])
goal_node = None # final node's data (cost, parent, ...) is uknown

In [308]:
def coord_valid(coords):
    check = coords + [row - 1 - coords[0], col - 1 - coords[1]]
    if sum(1 for num in check if num < 0) == 0:
        return True
    return False

def orc_available(coords):
    
    for orc in orc_data:
        if coords == orc[0:1]:
            return True
    return False

def guarded_by_orc(coords):
    
    for i in range(len(orc_data)):
        if (abs(coords[0]-orc_data[i][0]) + abs(coords[1]-orc_data[i][1])) <= orc_data[i][2]:
            return i + 1
    return -1

def member_is_in_state(current_state, member_state):
    
    for i in range(fellow_no):
        if current_state[i+2]==member_state:
            return True
    return False


def member_available(current_state):
    
    for i in range(fellow_no):
        if current_state[0:2] == fellow_strt[i] and current_state[2+i] == 2:
            return i+1
    return -1


def update_fellowship(current_state):
    
    if member_available(current_state) != -1 and not member_is_in_state(current_state, 1):
        
        member_available_number = member_available(current_state)
        current_state[1+member_available_number] = 1

    elif member_is_in_state(current_state, 1):
        
        for i in range(fellow_no):
            if current_state[0:2] == fellow_dest[i] and current_state[2+i] == 1:
                current_state[2+i] = 0
        
    return current_state

In [244]:
def extract_subnodes(current_node):
    
    subnodes = []
    
    up = [current_node.state[0]-1, current_node.state[1], 0]
    down = [current_node.state[0]+1, current_node.state[1], 1]
    right = [current_node.state[0], current_node.state[1]+1, 2]
    left = [current_node.state[0], current_node.state[1]-1, 3]
    
    actions = [up, down, right, left]
    
    i = 0
    while i < 4:
        
        action = actions[i]
        i += 1
        
        if action[0] < 0 or action[1] < 0 or action[0] >= row or action[1] >= col or orc_available(action[0:2]):
            continue
            
        next_state = update_fellowship(action[0:2] + current_node.state[2:])
        guarding_orc_no = guarded_by_orc(action[0:2])                

        if guarding_orc_no != -1:
            
            next_state[1 + fellow_no + guarding_orc_no] += 1
            
            if next_state[1 + fellow_no + guarding_orc_no] <= orc_data[guarding_orc_no-1][2]:
                subnodes.append(Node(next_state, current_node, current_node.path_cost + 1, current_node.path+[action[2]]))
        else:
            next_state[2 + fellow_no:] = [0] * orc_no
            subnodes.append(Node(next_state, current_node, current_node.path_cost + 1, current_node.path+[action[2]]))
            
            
    return subnodes

## Implementing the BFS Search Algorithm

In [245]:
def BFS(current_node, goal_state, print_path):
    
    frontier = [current_node]
    explored_and_frontier = set()
    states_no = 0
    
    while frontier:
        
        states_no += 1

        current_node = frontier.pop(0)
        explored_and_frontier.add(current_node.map)
        
        if current_node.state == goal_state:
            if print_path:
                print(current_node.path_str())
            return states_no
        
        next_nodes = extract_subnodes(current_node)
        
        for next_node in next_nodes:
            if next_node.map not in explored_and_frontier:
                frontier.append(next_node)
                explored_and_frontier.add(next_node.map)

In [309]:
print('paths:')

time0 = time.time()
BFS_number_of_states_3 = BFS(initial_node, goal_state, print_path=True)
time1 = time.time()
BFS_T1_3 = time1-time0

time0 = time.time()
BFS(initial_node, goal_state, print_path=False)
time1 = time.time()
BFS_T2_3 = time1-time0

time0 = time.time()
BFS(initial_node, goal_state, print_path=False)
time1 = time.time()
BFS_T3_3 = time1-time0

print('number of states is {}'.format(BFS_number_of_states))
BFS_mean_3 = mean([BFS_T1_3, BFS_T2_3, BFS_T3_3])
print('mean time consumed is {}'.format(BFS_mean_3))

paths:
DDRDRDDDDDLLDRURUUUUUUUURRRRRRRLLDDDDDLLDDDDLLLLURUUUURRRRRRRDDDDD
number of states is 7100
mean time consumed is 0.5533377329508463


## Implementing the IDS Search Algorithm

In [247]:
def DFS(current_node, goal_state, depth_limit):
    
    frontier = [current_node]
    explored_and_frontier = set()
    states_no = 0
    
    while frontier:
        
        states_no += 1
        
        current_node = frontier.pop()
        explored_and_frontier.add(current_node.map)
        
        if current_node.state == goal_state:
            return current_node, states_no

        next_nodes = extract_subnodes(current_node)
        
        for next_node in next_nodes:
            if next_node.map not in explored_and_frontier and depth_limit >= next_node.depth:
                frontier.append(next_node)
                explored_and_frontier.add(next_node.map)

In [248]:
def IDS(initial_node, goal_states, print_path):
    
    current_depth = 1
    
    while True:
        search_answer = DFS(initial_node, goal_state, current_depth)
        if search_answer is not None:
            goal_node, states_no = search_answer
            if print_path:
                print(goal_node.path_str())
            return states_no
        
        current_depth += 1

In [311]:
print('paths:')

time0 = time.time()
IDS_number_of_states_3 = IDS(initial_node, goal_state, print_path=True)
time1 = time.time()
IDS_T1_3 = time1-time0

time0 = time.time()
IDS(initial_node, goal_state, print_path=False)
time1 = time.time()
IDS_T2_3 = time1-time0

time0 = time.time()
IDS(initial_node, goal_state, print_path=False)
time1 = time.time()
IDS_T3_3 = time1-time0

print('number of states is {}'.format(IDS_number_of_states))
IDS_mean_3 = mean([IDS_T1_3, IDS_T2_3, IDS_T3_3])
print('mean time consumed is {}'.format(IDS_mean_3))

paths:
RRRRRRRLLLLLLLDDRRDRDRRRRRRDDDDDLLLLLLLLLRRRRRRRRRUUUULLLLLLLUULULUURRRRRRRRRLLLDDRDRRDDLLLLULLLDDDDRRRDRRRRUUUUULLLLLLLURDDLDDDLLRRRRRDRRRR
number of states is 1230
mean time consumed is 27.537041187286377


## Implementing A* Search Algorithm

### Heuristic Function

Since there is a possibility of visiting the same subnode from various parents (a node can be access from left, right, up, and down, if they are present), we are dealing with a graph search. As a result, our heuristic function should be consistent to have an optimal result.

Therefore our heuristic function should be:

h = manhattan distance between starting point and destination of members at starting point  (1)
    + manhattan distance between current place and destination of the member with Gandalf (if any) (2)
    + min dist between the current state (or the destination of the current member with Gandalf if any) to Gondor (3)
    + min dist between the destinations of members (4)


#### Consistency

the condition for being consistent is : Cost(A to C) + h(C) >= h(A)
                                        or  path_cost(C) - path_cost(A) + h(C) >= h(A)
                                        or  path_cost(C) - path_cost(A) >= h(A) - h(C)
                                        or real_cost(A to C) >= heuristic(A to C)
                                        
We prove the consistency by checking this equation for each bit of our heuristic function:

-(1),(2): the manhattan dist is the min path that Gandalf should pass and it can be longer due to the presence of orcs

-(3),(4): the smallest path is when the destination of each member is the starting point of another one and vice versa. In this condition, the closest starting point to current place plus the closest place from destinations to Gondor are to be added to (1) and (2) parts.

Therefore, we can conclude that each step towards our path has a real cost bigger than or equal to the heuristic function.

In [94]:
def manhattan(state1, state2):
    return (abs(state1[0] - state2[0]) + abs(state1[1] - state2[1]))

def dist(state1, state2):
    return math.sqrt(abs(state1[0] - state2[0])**2 + abs(state1[1] - state2[1])**2)

In [214]:
def A_star(print_path):
    
    frontier = [initial_node]
    frontier_fs = [initial_node]
    explored_and_frontier = set()
    
    states_no = 0
    
    while frontier:
        
        states_no += 1
        
        current_node = frontier.pop(0)
        explored_and_frontier.add(current_node.map)
        
        if current_node.state == goal_state:
            if print_path:
                print(current_node.path_str())
            return states_no
        
        next_nodes = extract_subnodes(current_node)
        
        
        for next_node in next_nodes:
            if next_node.map not in explored_and_frontier:
                bisect.insort(frontier, next_node)
                explored_and_frontier.add(next_node.map)
                

In [313]:
print('paths:')

time0 = time.time()
A_star_number_of_states_3 = A_star(print_path=True)
time1 = time.time()
A_star_T1_3 = time1-time0

time0 = time.time()
A_star(print_path=False)
time1 = time.time()
A_star_T2_3 = time1-time0

time0 = time.time()
A_star(print_path=False)
time1 = time.time()
A_star_T3_3 = time1-time0

print('number of states is {}'.format(A_star_number_of_states))
A_star_mean_3 = mean([A_star_T1_3, A_star_T2_3, A_star_T3_3])
print('mean time consumed is {}'.format(A_star_mean_3))

paths:
DDRDRDDDDDLLRDURUUUUUUUURRRRRRRLLDDDDDLLDDDDLLLLURUUUURRRRRRRDDDDD
number of states is 344
mean time consumed is 0.25337910652160645


In [316]:
print('paths:')

time0 = time.time()
A_star_number_of_states_3_W1 = A_star(print_path=True)
time1 = time.time()
A_star_T1_3_W1 = time1-time0

time0 = time.time()
A_star(print_path=False)
time1 = time.time()
A_star_T2_3_W1 = time1-time0

time0 = time.time()
A_star(print_path=False)
time1 = time.time()
A_star_T3_3_W1 = time1-time0

print('number of states is {}'.format(A_star_number_of_states))
A_star_mean_3_W1 = mean([A_star_T1_3_W1, A_star_T2_3_W1, A_star_T3_3_W1])
print('mean time consumed is {}'.format(A_star_mean_3_W1))

paths:
DDRDRDDDDDLLRDURUUUUUUUURRRRRRRLLDDDDDLLDDDDLLLLURUUUURRRRRRRDDDDD
number of states is 344
mean time consumed is 0.073333740234375


In [327]:
print('paths:')

time0 = time.time()
A_star_number_of_states_3_W2 = A_star(print_path=True)
time1 = time.time()
A_star_T1_3_W2 = time1-time0

time0 = time.time()
A_star(print_path=False)
time1 = time.time()
A_star_T2_3_W2 = time1-time0

time0 = time.time()
A_star(print_path=False)
time1 = time.time()
A_star_T3_3_W2 = time1-time0

print('number of states is {}'.format(A_star_number_of_states))
A_star_mean_3_W2 = mean([A_star_T1_3_W2, A_star_T2_3_W2, A_star_T3_3_W2])
print('mean time consumed is {}'.format(A_star_mean_3_W2))

paths:
RRRRRRRDDDDDLLDDDDLLLLUDURUUUUUUUURRRRRRRDLLLLULLDLDDDDDDDLLRRUUUURRRRRRRDDDDD
number of states is 344
mean time consumed is 0.14333359400431314


## Conclusion

### Table of Results

#### Test_00

In [238]:
data = {'SearchAlgorithm': ['BFS', 'IDS', 'A*', 'Weighted A*Weight (alpha=2.5)', 'Weighted A*Weight (alpha=1.5)'],
        'AnswerLength': [48, 88, 48, 48, 48],
        '# States':[BFS_number_of_states_0, IDS_number_of_states_0, A_star_number_of_states_0, A_star_number_of_states_0_W1, A_star_number_of_states_0_W2], 
        'MeanofConsumedTime': [BFS_mean_0, IDS_mean_0, A_star_mean_0, A_star_mean_0_W1, A_star_mean_0_W2]}  
       
df = pd.DataFrame(data)
df

,SearchAlgorithm,AnswerLength,# States,MeanofConsumedTime
0,BFS,48,7100,0.386208
1,IDS,88,1230,8.782216
2,A*,48,344,0.123377
3,Weighted A*Weight (alpha=2.5),48,172,0.076667
4,Weighted A*Weight (alpha=1.5),48,131,0.063335


#### Test_01

In [281]:
data = {'SearchAlgorithm': ['BFS', 'IDS', 'A*', 'Weighted A*Weight (alpha=2.5)', 'Weighted A*Weight (alpha=1.5)'],
        'AnswerLength': [52, 70, 52, 56, 56],
        '# States':[BFS_number_of_states_1, IDS_number_of_states_1, A_star_number_of_states_1, A_star_number_of_states_1_W1, A_star_number_of_states_1_W2], 
        'MeanofConsumedTime': [BFS_mean_1, IDS_mean_1, A_star_mean_1, A_star_mean_1_W1, A_star_mean_1_W2]}  
       
df = pd.DataFrame(data)
df

,SearchAlgorithm,AnswerLength,# States,MeanofConsumedTime
0,BFS,52,1821,0.119995
1,IDS,70,522,3.108636
2,A*,52,596,0.123331
3,Weighted A*Weight (alpha=2.5),56,253,0.059997
4,Weighted A*Weight (alpha=1.5),56,502,0.103330


#### Test_02

In [302]:
data = {'SearchAlgorithm': ['BFS', 'IDS', 'A*', 'Weighted A*Weight (alpha=2.5)', 'Weighted A*Weight (alpha=1.5)'],
        'AnswerLength': [34, 44, 34, 36, 34],
        '# States':[BFS_number_of_states_2, IDS_number_of_states_2, A_star_number_of_states_2, A_star_number_of_states_2_W1, A_star_number_of_states_2_W2], 
        'MeanofConsumedTime': [BFS_mean_2, IDS_mean_2, A_star_mean_2, A_star_mean_2_W1, A_star_mean_2_W2]}  
       
df = pd.DataFrame(data)
df

,SearchAlgorithm,AnswerLength,# States,MeanofConsumedTime
0,BFS,34,404,0.030001
1,IDS,44,132,0.286031
2,A*,34,150,0.019999
3,Weighted A*Weight (alpha=2.5),36,104,0.016667
4,Weighted A*Weight (alpha=1.5),34,114,0.023333


#### Teset_03

In [326]:
data = {'SearchAlgorithm': ['BFS', 'IDS', 'A*', 'Weighted A*Weight (alpha=2.5)', 'Weighted A*Weight (alpha=1.5)'],
        'AnswerLength': [66, 140, 66, 66, 78],
        '# States':[BFS_number_of_states_3, IDS_number_of_states_3, A_star_number_of_states_3, A_star_number_of_states_3_W1, A_star_number_of_states_3_W2], 
        'MeanofConsumedTime': [BFS_mean_3, IDS_mean_3, A_star_mean_3, A_star_mean_3_W1, A_star_mean_3_W2]}  
       
df = pd.DataFrame(data)
df

,SearchAlgorithm,AnswerLength,# States,MeanofConsumedTime
0,BFS,66,6635,0.553338
1,IDS,140,879,27.537041
2,A*,66,911,0.253379
3,Weighted A*Weight (alpha=2.5),66,272,0.073334
4,Weighted A*Weight (alpha=1.5),78,501,0.126667


#### Conclusiotn No. 1 - Iterative deepening depth first search is not optimal

There are conditions in graph searching that make the iddfs search not optimal. An example for this is:

- depth limit: 3
- Graph:

In this position Node C will be visited first from (A-B-C) path which is more expensive. As a result, with depth_limit == 3, reaching to our goal node (G) will be impossible if we start with spreading A to B.

Source: https://stackoverflow.com/questions/5001014/error-with-optimality-in-iterative-deepening-depth-first-search-algorithm

Note: The other two algoritms are complete and optimal

#### Conclusiotn No. 2 - Speed Comparison

The IDS algorithm is created of many DFSs. Though DFS is a bit faster than BFS by itself, repeatedly using it for various depths makes it far more slower than BFS.

A* algorithm, due to the usage of heuristic function, is more purposeful in choosing the states and therefore ends up exploring less of them. As a result A* becomes very faster than the other two uninformed search algorithms.

#### Conclusiotn No. 3 - Weighted A* is not optimal

As seen in tests 1 to 3, though A* speeds up after inflating the heuristic function, its output is no longer optimal. The reason for this is that the heuristic function is no longer admissible and therefore, not consistent. As a result, it does not guarantee that the output is optimal